# Scrollytelling_wine
---  

---  

In [2]:
# requirements
import pandas as pd

## Countries
---  

---

In [5]:
# country names
countries_df = pd.read_table("country_names.tsv")
countries_df.head()

,id,id_3char,name
0,afago,ago,Angola
1,afbdi,bdi,Burundi
2,afben,ben,Benin
3,afbfa,bfa,Burkina Faso
4,afbwa,bwa,Botswana


In [51]:
# which code is US?
countries_df[ countries_df['name'].str.contains("United States") ]

,id,id_3char,name
216,naumi,umi,United States Minor Outlying Islands
217,nausa,usa,United States


In [52]:
# which code is Spain?
countries_df[ countries_df['name'].str.contains("Spain") ]

,id,id_3char,name
135,euesp,esp,Spain


## Products
---  

---

In [9]:
# product names
products_df = pd.read_table("products_hs_07.tsv")
products_df[ products_df['name'].str.contains("Wine") ]

,id,hs07,name
851,42204,2204,"Wine of fresh grapes, including fortified wine..."
853,4220421,220421,Wine other than sparkling wine of fresh grapes...
854,4220429,220429,Wine other than sparkling wine of fresh grapes...
899,42307,2307,Wine lees; argol
900,4230700,230700,Wine lees; argol


In [10]:
# wine lees is not ov interest
wineHS = [2204, 220421, 220429]

## Exports
---  

---

In [43]:
# exports
exports_df = pd.read_table("year_origin_hs07_4.tsv")
exports_df = exports_df[ exports_df['hs07'].isin(wineHS) ].reset_index(drop=True)
exp2014_df = exports_df[exports_df['year'] == 2014].drop(columns=['import_val', 'export_rca', 'import_rca'])

# we have 218 data points
exp2014_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 1352 to 1569
Data columns (total 4 columns):
year          218 non-null int64
origin        218 non-null object
hs07          218 non-null int64
export_val    165 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 8.5+ KB


In [90]:
# what is total exports globally worth?
total = exp2014_df.export_val.sum()
print('$', round(total/1000000000,2),'Billion')

$ 34.98 Billion


In [83]:
# what portion was US and Spain?
usa = exp2014_df[ exp2014_df['origin'] == 'usa' ]['export_val'].values[0]
spain = exp2014_df[ exp2014_df['origin'] == 'esp' ]['export_val'].values[0]
print ('usa exports = ', round(usa/total*100, 2), '% of global exports')
print ('spain exports = ', round(spain/total*100,2), '% of global exports')

usa exports =  4.4 % of global exports
spain exports =  9.76 % of global exports


## Putting it together
---  

---

In [91]:
exp2014_df.head()

,year,origin,hs07,export_val
1352,2014,ago,2204,3079.6
1353,2014,bdi,2204,NaN
1354,2014,ben,2204,3013.0
1355,2014,bfa,2204,1204846.2
1356,2014,caf,2204,NaN


In [97]:
df = pd.merge(exp2014_df, countries_df, left_on='origin', right_on='id_3char').drop(columns=['origin', 'id', 'id_3char'])
df = pd.merge(df,products_df, on='hs07').drop(columns=['id'])
df.columns = ['year', 'hs_code', 'export_value_USD', 'country', 'product']
df.head()

,year,hs_code,export_value_USD,country,product
0,2014,2204,3079.6,Angola,"Wine of fresh grapes, including fortified wine..."
1,2014,2204,NaN,Burundi,"Wine of fresh grapes, including fortified wine..."
2,2014,2204,3013.0,Benin,"Wine of fresh grapes, including fortified wine..."
3,2014,2204,1204846.2,Burkina Faso,"Wine of fresh grapes, including fortified wine..."
4,2014,2204,NaN,Central African Republic,"Wine of fresh grapes, including fortified wine..."


In [99]:
df.to_json('exports2014.json', orient='records', lines=True)